# Evaluation on Disease Classification

In [ ]:
# https://www.youtube.com/watch?v=ac6ZB5QEwGU&ab_channel=DataTalksClub%E2%AC%9B
# https://huggingface.co/datasets/PardisSzah/DiseaseDecipher

In [1]:
import TestBot_for_Eval

In [2]:
import pandas as pd
from tqdm import tqdm
import random
from tqdm.asyncio import tqdm_asyncio
from sklearn.metrics import accuracy_score
from ollama import ChatResponse
import asyncio
import nest_asyncio

In [3]:
nest_asyncio.apply()

In [43]:
#query = "What is lung cancer"
#bot_response = TestBot_for_Eval.chatbot_response(query)
#bot_response

## EDA Test dataset

In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
df = pd.read_csv("documents/DiseaseDecipher.csv")

In [6]:
df.shape

(12736, 4)

In [7]:
df.columns

Index(['Question', 'Answer Disease', 'Answer Index', 'Number of Symptoms'], dtype='object')

In [8]:
df.tail()

,Question,Answer Disease,Answer Index,Number of Symptoms
12731,"A patient is presenting with the following symptoms: Eye redness, Pain in eye, Itchiness of eye, Fever, Swollen eye, Nasal congestion. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Arrhythmia 2) Skin cancer 3) Lateral epicondylitis (tennis elbow) 4) Conjunctivitis due to virus",Conjunctivitis due to virus,4,6
12732,"A patient is presenting with the following symptoms: Headache, Nosebleed, Facial pain, Hot flashes, Sleepiness, Sore in nose. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Open wound of the nose 2) Bone cancer 3) Ganglion cyst 4) Paroxysmal ventricular tachycardia",Open wound of the nose,1,6
12733,"A patient is presenting with the following symptoms: Headache, Nosebleed, Facial pain, Hot flashes, Sleepiness, Sore in nose. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Paroxysmal ventricular tachycardia 2) Open wound of the nose 3) Bone cancer 4) Ganglion cyst",Open wound of the nose,2,6
12734,"A patient is presenting with the following symptoms: Headache, Nosebleed, Facial pain, Hot flashes, Sleepiness, Sore in nose. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Bone cancer 2) Paroxysmal ventricular tachycardia 3) Open wound of the nose 4) Ganglion cyst",Open wound of the nose,3,6
12735,"A patient is presenting with the following symptoms: Headache, Nosebleed, Facial pain, Hot flashes, Sleepiness, Sore in nose. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Bone cancer 2) Paroxysmal ventricular tachycardia 3) Ganglion cyst 4) Open wound of the nose",Open wound of the nose,4,6


## Dataset intersection

### Idea: use only those diseases for testing which were in the database creation dataset, so QureAI should know about it

In [9]:
diseases = list(set(df["Answer Disease"].to_list()))

In [10]:
len(diseases)

796

In [11]:
df_medquad = pd.read_csv("documents/medquad.csv")

In [12]:
df_medquad.columns

Index(['question', 'answer', 'source', 'focus_area'], dtype='object')

In [13]:
diseases_medquad = list(df_medquad.focus_area.unique())

In [14]:
len(diseases_medquad)

5127

In [15]:
disease_intersection = [d for d in diseases if d in diseases_medquad]

In [16]:
len(disease_intersection)

147

In [17]:
disease_intersection[0]

'Down syndrome'

In [18]:
df.columns

Index(['Question', 'Answer Disease', 'Answer Index', 'Number of Symptoms'], dtype='object')

In [19]:
filtered_df = df[df['Answer Disease'].isin(disease_intersection)]

In [20]:
filtered_df = filtered_df.reset_index()

In [21]:
filtered_df.shape

(2352, 5)

In [22]:
filtered_df.head()

,index,Question,Answer Disease,Answer Index,Number of Symptoms
0,8,"A patient is presenting with the following symptoms: Groin mass, Leg pain, Hip pain. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Turner syndrome 2) Ulcerative colitis 3) Eye alignment disorder 4) Hyperlipidemia",Turner syndrome,1,3
1,9,"A patient is presenting with the following symptoms: Groin mass, Leg pain, Hip pain. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Hyperlipidemia 2) Turner syndrome 3) Eye alignment disorder 4) Ulcerative colitis",Turner syndrome,2,3
2,10,"A patient is presenting with the following symptoms: Groin mass, Leg pain, Hip pain. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Eye alignment disorder 2) Ulcerative colitis 3) Turner syndrome 4) Hyperlipidemia",Turner syndrome,3,3
3,11,"A patient is presenting with the following symptoms: Groin mass, Leg pain, Hip pain. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Eye alignment disorder 2) Hyperlipidemia 3) Ulcerative colitis 4) Turner syndrome",Turner syndrome,4,3
4,72,"A patient is presenting with the following symptoms: Diminished vision, Pain in eye, Symptoms of eye. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Glaucoma 2) Herpangina 3) G6PD enzyme deficiency 4) Pulmonary embolism",Glaucoma,1,3


In [23]:
filtered_df.to_csv('documents/filtered_test_data.csv', index=False)

## Select random eval sample

In [23]:
sample_size = 100
random.seed(42)
sample_ids = random.sample(range(len(filtered_df)), sample_size)

In [24]:
sample_questions = [filtered_df.loc[i, "Question"] for i in sample_ids]
sample_answers = [filtered_df.loc[i, "Answer Disease"] for i in sample_ids]

In [25]:
sample_questions[:4]

['A patient is presenting with the following symptoms: Fever, Cough, Sore throat. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Flu 2) Avascular necrosis 3) Mastectomy 4) Open wound of the lip',
 'A patient is presenting with the following symptoms: Skin rash, Skin lesion, Headache. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Bone disorder 2) Injury to the leg 3) Vasculitis 4) Pneumothorax',
 'A patient is presenting with the following symptoms: Eye redness, Diminished vision, Itchy scalp, Blindness. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Rosacea 2) Poisoning due to antihypertensives 3) Toxoplasmosis 4) High blood pressure',
 'A patient is presenting with the following symptoms: Weakness, Fatigue, Chills, Problems with movement. Based on these symptoms, which of the following diseases is the most likely diagnosis?\n1) Open wound due to trauma 2

In [26]:
len(sample_questions), len(sample_answers)

(100, 100)

## Eval without RAG

In [ ]:
# https://python.langchain.com/docs/integrations/chat/ollama/

In [32]:
import warnings
from langchain.prompts.prompt import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableConfig

In [35]:
!ollama list

NAME              ID              SIZE      MODIFIED   
mistral:latest    3944fe81ec14    4.1 GB    5 days ago    
llama3.2:1b       baf6a787fdff    1.3 GB    5 days ago    
deepseek-r1:8b    6995872bfe4c    5.2 GB    5 days ago    
llama3:latest     365c0bd3c000    4.7 GB    5 days ago    


In [ ]:
#!ollama pull "mistral"

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏  59 KB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏ 502 KB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏ 1.3 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏ 2.1 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏ 2.4 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea:   0% ▕                  ▏ 2.9 MB/4.1 G

In [28]:
warnings.filterwarnings("ignore")

In [29]:
#local_llm = "mistral"
local_llm = "llama3:latest"
#local_llm = "llama3.2:1b"

In [35]:
# Define LLM

# Local LLM:
llm = ChatOllama(
    model=local_llm,
    temperature=0,
    base_url="http://localhost:11434"
)

In [36]:
query = """
    Answer the following multiple-choice question. Respond only with the according disease without any preceding number or comment:\n\n
    {multiple_choice_question}
    """

In [37]:
prompt_template = PromptTemplate(
    input_variables=["multiple_choice_question"],
    template=query
)

In [38]:
chain = prompt_template | llm

#### SYNC:

In [41]:
llm_responses_pure = []
for question in tqdm(sample_questions):
    output = chain.invoke(input={"multiple_choice_question": question})
    llm_responses_pure.append(output)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:53<00:00,  1.88it/s]


#### ASYNC:

In [42]:
async def get_responses_batch(sample_questions):
    # Prepare the list of inputs for the chain
    batch_inputs = [{"multiple_choice_question": q} for q in sample_questions]
    
    # control concurrency
    config = RunnableConfig(max_concurrency=200) 
    
    llm_responses = await chain.abatch(batch_inputs, config=config)
    
    return llm_responses

async def get_responses(sample_questions):
    print("Starting concurrent LLM requests using batching...")
    llm_responses_pure = await get_responses_batch(sample_questions)
    print("All LLM responses received.")
    print(f"Received {len(llm_responses_pure)} responses.")
    return llm_responses_pure

llm_responses_pure = await get_responses(sample_questions)

Starting concurrent LLM requests using batching...
All LLM responses received.
Received 100 responses.


In [43]:
len(llm_responses_pure)

100

In [44]:
llm_pure_response_texts = [response.content for response in llm_responses_pure]

In [45]:
llm_pure_response_texts[:10]

['Flu',
 'Vasculitis',
 'Rosacea',
 'Multiple myeloma',
 'Psoriasis',
 'Deep Vein Thrombosis (DVT)',
 'Malaria',
 'Essential tremor',
 'Myocarditis',
 'Acid Reflux Disease']

### Eval score

In [46]:
y_true = sample_answers

In [47]:
len(y_true), len(llm_pure_response_texts)

(100, 100)

In [48]:
accuracy_score(y_true, llm_pure_response_texts)

0.74

In [ ]:
# DB: MedQuad q + full answers

# Without RAG
# local mistral: acc 0.6 + more talkative
# local llama3: acc 0.74

# WITH RAG
# local llama3: acc 0.94

In [ ]:
# DB: MedQuad q + full answers

# Pure local LLM
# llama3: acc 0.74

# QureAI:
# llama3: acc 0.94

In [49]:
for a,b in zip(y_true, llm_pure_response_texts):
    if not a==b:
        print(a)
        print(b)
        print("")
        print("------")
        print("")

Toxoplasmosis
Rosacea

------

Necrotizing fasciitis
Deep Vein Thrombosis (DVT)

------

Gastritis
Acid Reflux Disease

------

Polycystic kidney disease
Kidney Stone

------

Oral leukoplakia
Hemophiliac Lung Disease

------

Essential tremor
Huntington's disease

------

Emphysema
Myocardial Infarction

------

Myoclonus
Huntington's disease

------

Meningioma
Encephalitis

------

Anxiety
Hypercalcemia

------

Cystic Fibrosis
Lung Cancer

------

Thalassemia
Gastroesophageal Reflux Disease (GERD)

------

Leishmaniasis
Chronic rheumatic fever

------

Myositis
Guillain-Barré Syndrome

------

Emphysema
Myocardial Infarction

------

Dementia
Huntington's

------

Pseudohypoparathyroidism
Osteoarthritis

------

Priapism
Chlamydia

------

Aspergillosis
Aortic Dissection

------

Frostbite
Raynaud's disease

------

Pseudohypoparathyroidism
Osteoarthritis

------

Priapism
Chlamydia

------

Muscular dystrophy
Pneumonia

------

Muscular dystrophy
Sporotrichosis

------

Scabies
Pe

## Eval with RAG

In [50]:
task = "Answer the following multiple-choice question. Respond only with the according disease without any preceding number or comment: "

#### SYNC

In [49]:
index = 0
llm_responses_RAG = []
for question in tqdm(sample_questions):
    prompt = task + "\n\n" + question
    output = TestBot_for_Eval.chatbot_response(prompt)   
    llm_responses_RAG.append(output)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [17:42<00:00, 10.62s/it]


### Eval score

In [50]:
y_true = sample_answers

In [51]:
len(y_true), len(llm_responses_RAG)

(100, 100)

In [52]:
y_true_lower = [y.lower() for y in y_true]
llm_responses_RAG_lower = [resp.lower() for resp in llm_responses_RAG]

In [55]:
accuracy_score(y_true, llm_responses_RAG)
#accuracy_score(y_true_lower, llm_responses_RAG_lower)

0.87

In [ ]:

# EVALUATION ON FULL TEST DATASET

# Local Llama3:   acc 0.66
# QureAI:         acc 0.26
    

# EVALUATION ON TEST DATASET REDUCED TO DATABASE KNOWLEDGE

# Local Llama3:   acc 0.75
# QureAI:         acc 0.91


In [ ]:
# DB: MedQuad q + full answers


# EVALUATION ON FULL TEST DATASET

# Without RAG:
# llama model:                   acc 0.66

# With RAG:
# llama model, chunk_size ?:     acc 0.26



# EVALUATION ON TEST DATASET REDUCED TO MEDBOT-KNOWN DISEASES 

# Without RAG:
# llama model:                    acc 0.75

# With RAG:
# llama model, chunk_size 200:    acc 0.91
# llama model, chunk_size 2_000:  acc 0.94
# llama model, q + full answers:  acc 0.95

In [34]:
#for a,b in zip(y_true, llm_responses_RAG):
for a,b in zip(y_true_lower, llm_responses_RAG_lower):
    if not a==b:
        print(a)
        print(b)
        print("")

multiple sclerosis
chronic inflammatory demyelinating polyneuropathy

cystic fibrosis
lung cancer

frostbite
raynaud's disease

myoclonus
huntington's disease

lice
psoriasis

cholesteatoma
otitis media

thrombocytopenia
kidney cancer

multiple sclerosis
chronic inflammatory demyelinating polyneuropathy

pseudohypoparathyroidism
fibromyalgia

hemochromatosis
testicular cancer

thalassemia
pneumonia

kaposi sarcoma
peripheral artery disease

cystic fibrosis
pneumonia

scleroderma
psoriasis

myositis
rheumatoid arthritis

pemphigus
psoriasis

encephalitis
multiple sclerosis

hypoglycemia
sarcoidosis

tendinitis
fibromyalgia

marfan syndrome
gastroesophageal reflux disease (gerd)

marfan syndrome
irritable bowel syndrome (ibs)

autism
epilepsy

rabies
migraine

cataract
glaucoma

polycythemia vera
lupus

hemophilia
pulmonary embolism

achalasia
esophageal cancer

von willebrand disease
thrombocytopenia

muscular dystrophy
guillain-barré syndrome

emphysema
pneumonia

polycythemia vera
lup

#### ASYNC

In [56]:
from langchain_core.runnables import chain
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import warnings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts.prompt import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
import csv
import os

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from typing import List, Dict, Any
from langchain.memory import ChatMessageHistory

In [57]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [62]:

################################

# Hyperparameters

# threshold for similarity computation between user query and database vectors
similarity_threshold = 0.3

# max number of retrieved chunks (could be less depending on similarity scores and threshold)
k = 3


################################

warnings.filterwarnings("ignore")


# Local:
llm = ChatOllama(
    model=local_llm,
    temperature=0,
    # other params...
)

# === NEW: TQDM CALLBACK FOR BATCH PROCESSING ===
class TQDMCallback(BaseCallbackHandler):
    """A callback handler that updates a tqdm progress bar."""
    def __init__(self, total_tasks, description="Processing LLM Requests"):
        self.progress_bar = tqdm_asyncio(total=total_tasks, desc=description)

    def on_llm_end(self, response: BaseMessage, **kwargs: Any) -> None:
        """Callback for when the LLM finishes a call."""
        self.progress_bar.update(1)

    def __del__(self):
        """Ensure the progress bar is closed when the object is deleted."""
        self.progress_bar.close()


# === ASYNCHRONOUSLY-COMPATIBLE RAG PIPELINE ===

# Load embedded chunks from Vector Database 
async def aretrieve_from_vector_db(vector_db_path):
    """
    This function splits out a retriever object from a local vector database asynchronously.
    """
    embeddings = embedding_model
    
    vectorstore = await asyncio.to_thread(
        FAISS.load_local,
        folder_path=vector_db_path,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
    return vectorstore



# This helper function extracts the content field from a message object. 
# It is used to simplify the extraction of text from model outputs.
def get_msg_content(msg):
    return msg.content


# This system prompt instructs the model to reformulate the user’s question into a standalone question, 
# ensuring it can be understood without referencing the chat history.
contextualize_system_prompt = (
"""Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question which can be understood \
without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."""
)


contextualize_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_system_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])


contextualize_chain = (
    contextualize_prompt
    | llm
    | get_msg_content
)


qa_system_prompt = (
    "You are an assistant for any medical concerns. Answer the user's questions or queries based on the below context. " 
    "If the context doesn't contain any relevant information to the question or if the context is empty, "
    "do NOT make something up and just say 'Sorry, I don't know. Can you rephrase your concern?':"
    "\n\n"
    "###"
    "{context}"
    "###"
)


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)


qa_chain = (
    qa_prompt
    | llm
    | get_msg_content
)


# Define the overall chain the uses both the retrieved documents and the chat history to answer the question
# === THE MAIN CHAIN IS NOW ASYNCHRONOUS ===
@chain
async def history_aware_qa(input):
    # Rephrase the question if needed
    if input.get('chat_history'):
        #question = await contextualize_chain.ainvoke(input)   # with chat memory
        question = input['input']                       # without chat memory
    else:  
        question = input['input']
    

    # Return docs and relevance scores in the range [0, 1]. 0 is dissimilar, 1 is most similar.
    # could be empty, depending on the threshold
    similarity_results = await vectorstore.asimilarity_search_with_relevance_scores(question, k=k, score_threshold=similarity_threshold)
    if similarity_results:
        context, scores = zip(*similarity_results)
    else:   # no chunks similar enough
        context = ()

    retrieval_successful = bool(context)
   
    
    # Get the final answer
    final_response = await qa_chain.ainvoke({
        **input,
        "context": context
    })

    return (final_response, retrieval_successful)



chat_history_store = {}
def get_chat_history_for_session(session_id: str):
    if session_id not in chat_history_store:
        chat_history_store[session_id] = ChatMessageHistory()
    return chat_history_store[session_id]

qa_with_history_async = RunnableWithMessageHistory(
    history_aware_qa,
    get_chat_history_for_session,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# === THE MAIN EXECUTION FUNCTION ===
async def async_chatbot_response(prompts):

    print("Loading vector database...")
    global vectorstore


    db_name = "vector_databases/vector_db_med_quad_answers"
    #db_name = "vector_databases/vector_db_med_quad_answers_full"
    #db_name = "vector_databases/vector_db_med_quad_answers_chunk_size_500"
    #db_name = "vector_databases/vector_db_med_quad_answers_chunk_size_2000"
    
    vectorstore = await aretrieve_from_vector_db(db_name)
    print(f"Vector database {db_name} loaded.")

   
    print("Starting batch processing with a progress bar...")
    
    config = RunnableConfig(
        max_concurrency=200, 
        callbacks=[TQDMCallback(total_tasks=len(prompts))],
        configurable={"session_id": "dummy_session_id"} 
    )
    
    batch_inputs = [{
        "input": q, 
        "chat_history": [],
        "configurable": {"session_id": f"session_{i}"}
    } for i, q in enumerate(prompts)]
    
    results_with_status = await qa_with_history_async.abatch(batch_inputs, config=config)

    # === CRITICAL CHANGE: Aggregate the results after the batch is complete ===
    responses = [response for response, status in results_with_status]
    failed_retrievals = [status for response, status in results_with_status if not status]
    
    num_responses_without_retrievals = len(failed_retrievals)

    print("\nBatch processing complete.")
    print(f"Total responses without retrieval: {num_responses_without_retrievals}\n")
    
    
    return responses

In [63]:
prompts = [task + "\n\n" + question for question in sample_questions]

In [64]:
llm_responses_RAG = await async_chatbot_response(prompts)

Loading vector database...
Vector database vector_databases/vector_db_med_quad_answers loaded.
Starting batch processing with a progress bar...


Processing LLM Requests: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:30<00:00,  1.11it/s]


Batch processing complete.
Total responses without retrieval: 0



### Eval score

In [65]:
y_true = sample_answers

In [66]:
len(y_true), len(llm_responses_RAG)

(100, 100)

In [67]:
accuracy_score(y_true, llm_responses_RAG)

0.71

In [134]:
llm_responses_RAG.count("Sorry, I don't know. Can you rephrase your concern?")

232

In [106]:
for a,b in zip(y_true, llm_responses_RAG):
    if not a==b:
        print(a)
        print(b)
        print("")

Turner syndrome
Hernia

Turner syndrome
Hernia

Turner syndrome
Femoral Hernia

Congenital rubella
Benign paroxysmal positional vertical (BPPV)

Congenital rubella
BPPV

Congenital rubella
Benign paroxysmal positional vertical (BPPV)

Congenital rubella
Benign paroxysmal positional vertical (BPPV)

Emphysema
Myocardial Infarction

Emphysema
Myocardial Infarction

Emphysema
Acute Coronary Syndrome

Emphysema
Myocardial Infarction

Cholesteatoma
Otitis Media

Obesity
Sinus bradycardia

Obesity
Sinus bradycardia

Obesity
Sinus bradycardia

Pericarditis
Myocardial Infarction

Lice
Scabies

Lice
Scabies

Oral leukoplakia
Hemophiliac Lung Disease

Priapism
Chlamydia

Priapism
Urinary Tract Infection (UTI)

Polycystic kidney disease
Kidney Stone

Polycystic kidney disease
Kidney Stone

Amyloidosis
Anxiety

Amyloidosis
Pulmonary Embolism

Scleroderma
Eczema

Lymphoma
Syphilis

Lymphoma
Syphilis

Hepatic encephalopathy
Hepatic Encephalopathy

Leishmaniasis
Chronic rheumatic fever

Leishmaniasis